In [1]:
import tensorflow as tf

2024-06-05 13:00:34.929176: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)


training_set = train_datagen.flow_from_directory(
        'Train',
        target_size=(64, 64),
        batch_size=32, 
        class_mode='binary')

Found 10000 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'Test', 
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary') 

Found 992 images belonging to 2 classes.


In [46]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_set = test_datagen.flow_from_directory(
        'Validation', 
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary') 

Found 800 images belonging to 2 classes.


In [47]:
cnn = tf.keras.models.Sequential()

In [50]:
# Convolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64, 64, 3]))

In [51]:
# Pooling Layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [52]:
# Convolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))

In [53]:
# Pooling Layer
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [54]:
# Flattening
cnn.add(tf.keras.layers.Flatten())

In [55]:
# Full connection
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [56]:
# Output layer
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [57]:
# Compile
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [58]:
save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.keras',monitor='val_loss',verbose=1) 

In [60]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", #val_accuracy - for classification , val_loss - for regression
    min_delta=0.0001,
    patience=3,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
) 

In [61]:
cnn.fit(x=training_set,epochs=10,validation_data=validation_set,callbacks=[save_best_model,early_stopping])

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.8691 - loss: 0.2794

/opt/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: saving model to best_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 56s 168ms/step - accuracy: 0.8693 - loss: 0.2790 - val_accuracy: 0.9850 - val_loss: 0.0413
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.9792 - loss: 0.0603
Epoch 2: saving model to best_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 60s 191ms/step - accuracy: 0.9792 - loss: 0.0603 - val_accuracy: 0.9900 - val_loss: 0.0286
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step - accuracy: 0.9855 - loss: 0.0393
Epoch 3: saving model to best_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 63s 198ms/step - accuracy: 0.9855 - loss: 0.0393 - val_accuracy: 0.9862 - val_loss: 0.0415
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - accuracy: 0.9879 - loss: 0.0354
Epoch 4: saving model to best_model.keras
313/313 ━━━━━━━━━━━━━━━━━━━━ 54s 171ms/step - accuracy: 0.9879 - loss: 0.0354 - val_accuracy: 0.9937 - val_loss: 0.0199
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step - accuracy: 0.9890 - loss: 0.0314

In [70]:
import numpy as np
from tensorflow.keras.preprocessing import image

test_image = image.load_img('Prediction/3.png',target_size = (64, 64))
# test_image = image.load_img('Test/WithoutMask/1.png',target_size = (64, 64))
# test_image = image.load_img('Test/WithMask/219.png',target_size = (64, 64))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
result
training_set.class_indices
 
if result[0][0] == 1:
  prediction = 'Not Have Mask'
else:
  prediction = 'Have Mask'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [71]:
print(prediction)

Not Have Mask


In [77]:
test_loss, test_accuracy = cnn.evaluate(test_set)
print(f'Test accuracy: {test_accuracy:.2f}') 

31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 98ms/step - accuracy: 0.9902 - loss: 0.0230
Test accuracy: 0.99


In [75]:
tf.keras.models.save_model(cnn,'Face Mask Detection Model.h5') 